In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
import re

regex = re.compile(r'^[\d]+_x=([-\d.]+)_z=([-\d.]+)_a=([-\d.]+)_h=([-\d.]+)[.]csv$')

def label_from_path(path):
    label = path.split('\\')[-1] # PLATFORM DEPENDENT!
    m = regex.match(label)
    out = [float(x) for x in m.groups()]
    return torch.tensor(out, dtype=torch.float)

In [3]:
def data_from_path(path):
    data = pd.read_csv(path, sep=';')

    time = data.values[:,0]
    vx   = data.values[:,1::2]
    vz   = data.values[:,2::2]
    vabs = (vx ** 2 + vz ** 2) ** (1/2)
    
    vx   = torch.from_numpy(vx)
    vz   = torch.from_numpy(vz)
    vabs = torch.from_numpy(vabs)
    
    vx_fft   = torch.view_as_real(torch.fft.fft(vx)).permute(2, 0, 1)
    vz_fft   = torch.view_as_real(torch.fft.fft(vz)).permute(2, 0, 1)
    vabs_fft = torch.view_as_real(torch.fft.fft(vabs)).permute(2, 0, 1)

    ret = torch.cat([vx.unsqueeze(0), vz.unsqueeze(0), vabs.unsqueeze(0), vx_fft, vz_fft, vabs_fft], dim=0)

    return ret

In [4]:
class CSVDataset(Dataset):
    def __init__(self, path, device):
        self.device = device
        self.csv_list = [
            (path + "\\" + i) for i in os.listdir(path) if i.split('.')[-1] == 'csv'
        ]

    def __getitem__(self, item):
        d = data_from_path(self.csv_list[item]).to(device=device, dtype=torch.float)
        l = label_from_path(self.csv_list[item]).to(device=device, dtype=torch.float)
        return d, l

    def __len__(self):
        return len(self.csv_list)

In [5]:
class Predictor(nn.Module):
    def __init__(self, n_hyper):
        super(Predictor, self).__init__()

        self.feature_extractor = nn.Sequential(
            #        in  out  ker size
            nn.Conv2d(9, n_hyper, 3),     # 9 = len([v_x, v_y, v_abs, 2*fft_vx, 2*fft_vy, 2*fft_vabs, ])
            nn.MaxPool2d(2, 2), 
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(n_hyper),

            nn.Conv2d(n_hyper, 512, 3, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),

            nn.Conv2d(512, 1024, 3, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(1024),

            nn.Conv2d(1024, 512, 2),
        )

        self.linear = nn.Linear(45, 256)
        self.conv = nn.Sequential(
            nn.Conv2d(512, 1, 1),
            nn.ReLU(inplace=True)
            )

        self.linear2 = nn.Linear(256, 4) # 4 = x, z, a, h

    def forward(self, input):
        x = self.feature_extractor(input)
        x = x.permute(0, 1, 3, 2)
        x = self.linear(x)
        x = self.conv(x)
        x = x.squeeze(1).squeeze(1)
        x = self.linear2(x)

        return x

In [6]:
loss_fn = nn.MSELoss()
predictor = Predictor(64).to(device)
predictor.load_state_dict(torch.load(".\\w3\\lastest_22_478.74707580566405"))

In [13]:
path = ".\\csv_test\\4_x=-1714.972192_z=1142.761271_a=91.818242_h=298.590158.csv"
data = data_from_path(path).to(device=device, dtype=torch.float)
label = label_from_path(path).to(device=device, dtype=torch.float)
print(label)

outputs = predictor(data.unsqueeze(0))
print(outputs)

tensor([-1714.9722,  1142.7612,    91.8182,   298.5901], device='cuda:0')
tensor([[-1752.2119,  1131.2064,    88.8645,   287.1279]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [15]:
path = ".\\csv_test\\3_x=214.705816_z=937.950430_a=84.379035_h=166.219929.csv"
data = data_from_path(path).to(device=device, dtype=torch.float)
label = label_from_path(path).to(device=device, dtype=torch.float)
print(label)

outputs = predictor(data.unsqueeze(0))
print(outputs)

tensor([214.7058, 937.9504,  84.3790, 166.2199], device='cuda:0')
tensor([[143.5333, 936.1996,  73.9638, 177.2107]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [16]:
path = ".\\csv_test\\2_x=868.422572_z=960.444919_a=76.251414_h=258.932863.csv"
data = data_from_path(path).to(device=device, dtype=torch.float)
label = label_from_path(path).to(device=device, dtype=torch.float)
print(label)

outputs = predictor(data.unsqueeze(0))
print(outputs)

tensor([868.4225, 960.4449,  76.2514, 258.9329], device='cuda:0')
tensor([[865.9625, 978.5985,  79.4575, 230.9267]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [17]:
path = ".\\csv_test\\1_x=961.429279_z=1371.068986_a=89.714692_h=242.039323.csv"
data = data_from_path(path).to(device=device, dtype=torch.float)
label = label_from_path(path).to(device=device, dtype=torch.float)
print(label)

outputs = predictor(data.unsqueeze(0))
print(outputs)

tensor([ 961.4293, 1371.0690,   89.7147,  242.0393], device='cuda:0')
tensor([[ 986.1116, 1384.7716,   84.6249,  228.8689]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
